In [2]:
import pandas as pd
import numpy as np 
from scipy.spatial import distance
import matplotlib.pyplot as plt
import scipy.stats as stats
import glob
import os
import math
%matplotlib inline

In [3]:
dir = 'D:\\CSCI_535\\project\\Audio_features\\'
segment_file = 'segment_annotation.csv'
basis_df = pd.read_csv(dir + segment_file)
start_segment = basis_df['start_segment_s']
end_segment = basis_df['end_segment_s']

In [4]:
features = ['pcm_fftMag_mfcc[0]',
 'pcm_fftMag_mfcc[1]',
 'pcm_fftMag_mfcc[2]',
 'pcm_fftMag_mfcc[3]',
 'pcm_fftMag_mfcc[4]',
 'pcm_fftMag_mfcc[5]',
 'pcm_fftMag_mfcc[6]',
 'pcm_fftMag_mfcc[7]',
 'pcm_fftMag_mfcc[8]',
 'pcm_fftMag_mfcc[9]',
 'pcm_fftMag_mfcc[10]',
 'pcm_fftMag_mfcc[11]',
 'pcm_fftMag_mfcc[12]',
 'pcm_fftMag_mfcc_de[0]',
 'pcm_fftMag_mfcc_de[1]',
 'pcm_fftMag_mfcc_de[2]',
 'pcm_fftMag_mfcc_de[3]',
 'pcm_fftMag_mfcc_de[4]',
 'pcm_fftMag_mfcc_de[5]',
 'pcm_fftMag_mfcc_de[6]',
 'pcm_fftMag_mfcc_de[7]',
 'pcm_fftMag_mfcc_de[8]',
 'pcm_fftMag_mfcc_de[9]',
 'pcm_fftMag_mfcc_de[10]',
 'pcm_fftMag_mfcc_de[11]',
 'pcm_fftMag_mfcc_de[12]',
 'pcm_fftMag_mfcc_de_de[0]',
 'pcm_fftMag_mfcc_de_de[1]',
 'pcm_fftMag_mfcc_de_de[2]',
 'pcm_fftMag_mfcc_de_de[3]',
 'pcm_fftMag_mfcc_de_de[4]',
 'pcm_fftMag_mfcc_de_de[5]',
 'pcm_fftMag_mfcc_de_de[6]',
 'pcm_fftMag_mfcc_de_de[7]',
 'pcm_fftMag_mfcc_de_de[8]',
 'pcm_fftMag_mfcc_de_de[9]',
 'pcm_fftMag_mfcc_de_de[10]',
 'pcm_fftMag_mfcc_de_de[11]',
 'pcm_fftMag_mfcc_de_de[12]\n',
 'chroma[0]',
 'chroma[1]',
 'chroma[2]',
 'chroma[3]',
 'chroma[4]',
 'chroma[5]',
 'chroma[6]',
 'chroma[7]',
 'chroma[8]',
 'chroma[9]',
 'chroma[10]',
 'chroma[11]\n',
 'chroma[0].1',
 'chroma[1].1',
 'chroma[2].1',
 'chroma[3].1',
 'chroma[4].1',
 'chroma[5].1',
 'chroma[6].1',
 'chroma[7].1',
 'chroma[8].1',
 'chroma[9].1',
 'chroma[10].1',
 'chroma[11]\n.1',
 'pcm_LOGenergy\n',
 'voiceProb_sma',
 'F0_sma',
 'pcm_loudness_sma\n',
 'F0final_sma',
 'voicingFinalUnclipped_sma',
 'pcm_loudness_sma\n.1']

In [5]:
comedian_means = pd.DataFrame()
comedian_std = pd.DataFrame()
comedians_list = []
for file in glob.glob(dir+"\\OpenSmile\\"'*.csv'):
    comedian = os.path.basename(file).split('_')[0]
    comedians_list.append(comedian)
    #'''
    comedian_audio_features_df = pd.read_csv(file)
    comedian_segment_df = basis_df[basis_df.video == comedian]
    start_segment = comedian_segment_df['start_segment_s']
    end_segment = comedian_segment_df['end_segment_s']
    #print(comedian_segment_df)
    
    comedian_audio_features_norm_df = comedian_audio_features_df[features]
    comedian_audio_features_norm_df = (comedian_audio_features_norm_df - comedian_audio_features_norm_df.min())/(comedian_audio_features_norm_df.max()-comedian_audio_features_norm_df.min())
    comedian_audio_features_norm_df['frameTime'] = comedian_audio_features_df['frameTime']
    
    seg_dfs = []
    for index in start_segment.index:
        tdf = comedian_audio_features_norm_df[(comedian_audio_features_norm_df.frameTime > int(start_segment[index])) & (comedian_audio_features_norm_df.frameTime < int(end_segment[index])) ]
        seg_dfs.append(tdf)
    
    segment_means = []
    segment_std = []
    for segment in seg_dfs:
        segment_means.append(segment[features].mean())
        segment_std.append(segment[features].std())
        
    segment_mean_values = pd.DataFrame(segment_means,index = comedian_segment_df.index)
    segment_std_values = pd.DataFrame(segment_std,index = comedian_segment_df.index)
    
    comedian_segment_mean = pd.concat([comedian_segment_df,segment_mean_values],axis=1)
    comedian_segment_std = pd.concat([comedian_segment_df,segment_std_values],axis=1)
        
    comedian_means = comedian_means.append(comedian_segment_mean)
    comedian_std = comedian_std.append(comedian_segment_std)
    
p_comedian_means = comedian_means[np.isfinite(comedian_means['F0_sma'])]
p_comedian_std = comedian_std[np.isfinite(comedian_std['F0_sma'])]

In [6]:
writer = pd.ExcelWriter(dir + "open_face_accoustic_means.xlsx")
comedian_means.to_excel(writer,'Sheet1')
writer.save()

In [7]:
writer = pd.ExcelWriter(dir + "open_face_accoustic_std.xlsx")
comedian_std.to_excel(writer,'Sheet1')
writer.save()

In [9]:
cv = list(comedian_means.video)
cvs = list(comedian_means.laughter_start)
c_v_vs = zip(cv,cvs)
pcv = list(p_comedian_means.video)
pcvs = list(p_comedian_means.laughter_start)
p_v_vs = zip(pcv,pcvs)
a = set(list(c_v_vs))
b = set(list(p_v_vs))
sorted(list(a - b))

[('AparnaNancherla1', 110.0),
 ('CristelaAlonza1', 0.0),
 ('MaryMack', 197.0),
 ('MaryMack', 208.0),
 ('MaryMack', 226.0),
 ('MaryMack', 240.0),
 ('MaryMack', 257.0),
 ('MaryMack', 264.0),
 ('MaryMack', 269.0),
 ('MaryMack', 283.0),
 ('MaryMack', 294.0),
 ('MaryMack', 300.0),
 ('MaryMack', 306.0),
 ('MaryMack', 322.0),
 ('MaryMack', 330.0),
 ('MeganGailey', 0.0),
 ('RussellHoward1', 43.0)]

In [19]:
comedian_means[comedian_means.video=='RussellHoward1']

,video,laughter_start,laughter_end,laughter_value,start_segment_s,end_segment_s,start_segment_frame,end_segment_frame,pcm_fftMag_mfcc[0],pcm_fftMag_mfcc[1],...,chroma[9].1,chroma[10].1,chroma[11] .1,pcm_LOGenergy,voiceProb_sma,F0_sma,pcm_loudness_sma,F0final_sma,voicingFinalUnclipped_sma,pcm_loudness_sma .1
1096,RussellHoward1,6.0,7.0,2,0.0,6.0,1,180,0.528606,0.519436,...,0.124332,0.151502,0.166566,0.841605,0.604755,0.290393,0.162532,0.274035,0.786471,0.162500
1097,RussellHoward1,13.0,14.0,2,7.0,13.0,211,390,0.543074,0.458746,...,0.135348,0.158101,0.167546,0.920131,0.719600,0.421737,0.294392,0.370760,0.857722,0.293595
1098,RussellHoward1,22.0,23.0,2,14.0,22.0,421,660,0.507384,0.417872,...,0.136329,0.168869,0.176618,0.914071,0.647336,0.352821,0.278693,0.400991,0.848074,0.278420
1099,RussellHoward1,28.0,29.0,2,23.0,28.0,691,840,0.556579,0.532056,...,0.150971,0.168847,0.181404,0.881271,0.678048,0.320353,0.217298,0.272305,0.819127,0.217466
1100,RussellHoward1,37.0,38.0,2,29.0,37.0,871,1110,0.512973,0.448393,...,0.133642,0.170015,0.171074,0.912488,0.666338,0.345860,0.289745,0.332095,0.840849,0.289878
1101,RussellHoward1,41.0,43.0,2,38.0,41.0,1141,1230,0.443880,0.358789,...,0.146152,0.185695,0.147567,0.921182,0.640388,0.376684,0.313057,0.444036,0.847229,0.313203
1102,RussellHoward1,43.0,45.0,2,43.0,43.0,1291,1290,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1103,RussellHoward1,47.0,49.0,2,45.0,47.0,1351,1410,0.428464,0.377423,...,0.130835,0.158543,0.150613,0.934090,0.646037,0.398890,0.342514,0.428312,0.841045,0.343516
1104,RussellHoward1,73.0,74.0,2,49.0,73.0,1471,2190,0.494391,0.483010,...,0.132731,0.162615,0.182814,0.883495,0.597606,0.305265,0.223825,0.319369,0.804635,0.223855
1105,RussellHoward1,80.0,81.0,2,74.0,80.0,2221,2400,0.576042,0.502898,...,0.161875,0.196304,0.184493,0.894776,0.680397,0.321777,0.230987,0.274761,0.812573,0.230595


In [8]:
len(list(p_comedian_means.video))

1318

In [14]:
basis_features = list(basis_df.columns)

In [17]:
comedian_means =  comedian_means.sort_values(['video','laughter_start'])[basis_features + features]
comedian_std =  comedian_std.sort_values(['video','laughter_start'])[basis_features + features]

In [9]:
1360-1318

42

In [263]:

low_l = []
medium_l = []
high_l = []
feature_l = []
low_std_l = []
medium_std_l = []
high_std_l = []

p_value_l = []
for feature in features:
    
    low = list(comedian_means[p_comedian_means.laughter_value == 0][feature])
    medium = list(comedian_means[p_comedian_means.laughter_value == 1][feature])
    high = list(comedian_means[p_comedian_means.laughter_value == 2][feature])
   
    a = stats.f_oneway(low,medium,high)
    
    if a[1] <= 0.05:
        low_l.append(np.mean(low))
        medium_l.append(np.mean(medium))
        high_l.append(np.mean(high))
        low_std_l.append(np.std(low))
        medium_std_l.append(np.std(medium)) 
        high_std_l.append(np.std(high))        
        p_value_l.append(a[1])
        feature_l.append(feature)

        data = [low,medium,high]
        labels = ['low','medium','high']
        plt.ylabel(feature)
        plt.boxplot(data,labels= labels)
        feature = feature.strip()
        feature = feature.replace("\n","")
        feature  = feature.replace(".","_")
        print(feature)
        plt.savefig(dir + "box_plot_mean\\" + feature.strip() + '_mean')
        plt.close()
    
    
    #print('*'*15)
p_value_result_mean_df = pd.DataFrame({
        'Feature' : pd.Series(feature_l),
        'Low_mean' : pd.Series(low_l),
        'Medium_mean':pd.Series(medium_l),
        'High_mean' : pd.Series(high_l),
        'Low_std' : pd.Series(low_std_l),
        'Medium_std': pd.Series(medium_std_l),
        'High_std' : pd.Series(high_std_l),
        'p_value' : pd.Series(p_value_l)
    })

pcm_fftMag_mfcc[0]
pcm_fftMag_mfcc[1]
pcm_fftMag_mfcc[2]
pcm_fftMag_mfcc[3]
pcm_fftMag_mfcc[4]
pcm_fftMag_mfcc[5]
pcm_fftMag_mfcc[6]
pcm_fftMag_mfcc[7]
pcm_fftMag_mfcc[8]
pcm_fftMag_mfcc[9]
pcm_fftMag_mfcc[10]
pcm_fftMag_mfcc[11]
pcm_fftMag_mfcc[12]
pcm_fftMag_mfcc_de[0]
pcm_fftMag_mfcc_de[1]
pcm_fftMag_mfcc_de[2]
pcm_fftMag_mfcc_de[3]
pcm_fftMag_mfcc_de[4]
pcm_fftMag_mfcc_de[5]
pcm_fftMag_mfcc_de[6]
pcm_fftMag_mfcc_de[7]
pcm_fftMag_mfcc_de[8]
pcm_fftMag_mfcc_de[9]
pcm_fftMag_mfcc_de[10]
pcm_fftMag_mfcc_de[11]
pcm_fftMag_mfcc_de[12]
pcm_fftMag_mfcc_de_de[0]
pcm_fftMag_mfcc_de_de[1]
pcm_fftMag_mfcc_de_de[2]
pcm_fftMag_mfcc_de_de[3]
pcm_fftMag_mfcc_de_de[4]
pcm_fftMag_mfcc_de_de[5]
pcm_fftMag_mfcc_de_de[6]
pcm_fftMag_mfcc_de_de[7]
pcm_fftMag_mfcc_de_de[8]
pcm_fftMag_mfcc_de_de[9]
pcm_fftMag_mfcc_de_de[10]
pcm_fftMag_mfcc_de_de[12]
chroma[0]
chroma[1]
chroma[2]
chroma[3]
chroma[4]
chroma[5]
chroma[6]
chroma[7]
chroma[8]
chroma[9]
chroma[10]
chroma[11]
chroma[0]_1
chroma[1]_1
chroma[2]_1
c

In [247]:
p_value_result_df.shape

(58, 8)

In [270]:
low_l = []
medium_l = []
high_l = []
feature_l = []
low_std_l = []
medium_std_l = []
high_std_l = []

p_value_l = []
for feature in features:
    
    low = list(comedian_std[comedian_std.laughter_value == 0][feature])
    medium = list(comedian_std[comedian_std.laughter_value == 1][feature])
    high = list(comedian_std[comedian_std.laughter_value == 2][feature])
    
    
    
    #print(np.mean(low))
    #print(np.mean(medium))
    #print(np.mean(high))
    
    a = stats.f_oneway(low,medium,high)
    
    if a[1] <= 0.05:
        low_l.append(np.mean(low))
        medium_l.append(np.mean(medium))
        high_l.append(np.mean(high))
        low_std_l.append(np.std(low))
        medium_std_l.append(np.std(medium)) 
        high_std_l.append(np.std(high))        
        p_value_l.append(a[1])
        feature_l.append(feature)

        data = [low,medium,high]
        labels = ['low','medium','high']
        plt.ylabel(feature)
        plt.boxplot(data,labels= labels)
        feature = feature.strip()
        feature = feature.replace("\n","")
        feature  = feature.replace(".","_")
        print(feature)
        plt.savefig(dir + "box_plot_std\\" + feature.strip() + '_std')
        plt.close()
    
    #print('*'*15)
p_value_result_std_df = pd.DataFrame({
        'Feature' : pd.Series(feature_l),
        'Low_mean' : pd.Series(low_l),
        'Medium_mean':pd.Series(medium_l),
        'High_mean' : pd.Series(high_l),
        'Low_std' : pd.Series(low_std_l),
        'Medium_std': pd.Series(medium_std_l),
        'High_std' : pd.Series(high_std_l),
        'p_value' : pd.Series(p_value_l)
    })

pcm_fftMag_mfcc[0]
pcm_fftMag_mfcc[1]
pcm_fftMag_mfcc[2]
pcm_fftMag_mfcc[3]
pcm_fftMag_mfcc[4]
pcm_fftMag_mfcc[5]
pcm_fftMag_mfcc[6]
pcm_fftMag_mfcc[7]
pcm_fftMag_mfcc[8]
pcm_fftMag_mfcc[9]
pcm_fftMag_mfcc[10]
pcm_fftMag_mfcc[11]
pcm_fftMag_mfcc[12]
pcm_fftMag_mfcc_de[0]
pcm_fftMag_mfcc_de[1]
pcm_fftMag_mfcc_de[2]
pcm_fftMag_mfcc_de[3]
pcm_fftMag_mfcc_de[4]
pcm_fftMag_mfcc_de[5]
pcm_fftMag_mfcc_de[7]
pcm_fftMag_mfcc_de[8]
pcm_fftMag_mfcc_de[9]
pcm_fftMag_mfcc_de[10]
pcm_fftMag_mfcc_de[11]
pcm_fftMag_mfcc_de_de[0]
pcm_fftMag_mfcc_de_de[1]
pcm_fftMag_mfcc_de_de[2]
pcm_fftMag_mfcc_de_de[3]
pcm_fftMag_mfcc_de_de[4]
pcm_fftMag_mfcc_de_de[5]
pcm_fftMag_mfcc_de_de[6]
pcm_fftMag_mfcc_de_de[7]
pcm_fftMag_mfcc_de_de[8]
pcm_fftMag_mfcc_de_de[10]
pcm_fftMag_mfcc_de_de[11]
pcm_fftMag_mfcc_de_de[12]
chroma[1]
chroma[2]
chroma[4]
chroma[5]
chroma[6]
chroma[7]
chroma[8]
chroma[9]
chroma[10]
chroma[11]
chroma[2]_1
chroma[4]_1
chroma[5]_1
chroma[6]_1
chroma[7]_1
chroma[8]_1
pcm_LOGenergy
voiceProb_sma
F

In [249]:
p_value_result_std_df.shape

(59, 8)

In [18]:
writer = pd.ExcelWriter(dir + "open_face_accoustic_means.xlsx")
comedian_means.to_excel(writer,'Sheet1')
writer.save()

In [19]:
writer = pd.ExcelWriter(dir + "open_face_accoustic_std.xlsx")
comedian_std.to_excel(writer,'Sheet1')
writer.save()

In [252]:
writer = pd.ExcelWriter(dir + "open_face_p_value_accoustic_std.xlsx")
p_value_result_std_df.to_excel(writer,'Sheet1')
writer.save()

In [253]:
writer = pd.ExcelWriter(dir + "open_face_p_value_accoustic_mean.xlsx")
p_value_result_mean_df.to_excel(writer,'Sheet1')
writer.save()

In [181]:
comedian_means[comedian_means.video=='MatthewBroussard'][['start_segment_s','end_segment_s','F0_sma']]

,start_segment_s,end_segment_s,F0_sma
938,0.0,4.0,145.618943
939,5.0,8.0,107.896703
940,9.0,11.0,93.486175
941,12.0,15.0,85.663403
942,16.0,20.0,72.904618
943,22.0,27.0,93.856392
944,28.0,31.0,61.965054
945,37.0,39.0,117.381203
946,40.0,48.0,123.791372
947,49.0,53.0,93.066134


In [112]:
segment_columns = ['video', 'laughter_start', 'laughter_end', 'laughter_value',
       'start_segment_s', 'end_segment_s', 'start_segment_frame',
       'end_segment_frame']

In [114]:
len(comedian_means.video.unique())

43

In [10]:
sample_df = pd.read_csv("D:\\CSCI_535\\project\\Audio_features\\AndyWoodhull2_Audio.wav.csv.csv")

In [32]:
andy_df = basis_df[basis_df.video == 'AndyWoodhull2']
start_segment = andy_df['start_segment_s']
end_segment = andy_df['end_segment_s']

In [16]:
tdf = sample_df[(sample_df.frameTime > 0) & (sample_df.frameTime < 3) ]

In [86]:
features = ['pcm_fftMag_mfcc[0]',
 'pcm_fftMag_mfcc[1]',
 'pcm_fftMag_mfcc[2]',
 'pcm_fftMag_mfcc[3]',
 'pcm_fftMag_mfcc[4]',
 'pcm_fftMag_mfcc[5]',
 'pcm_fftMag_mfcc[6]',
 'pcm_fftMag_mfcc[7]',
 'pcm_fftMag_mfcc[8]',
 'pcm_fftMag_mfcc[9]',
 'pcm_fftMag_mfcc[10]',
 'pcm_fftMag_mfcc[11]',
 'pcm_fftMag_mfcc[12]',
 'pcm_fftMag_mfcc_de[0]',
 'pcm_fftMag_mfcc_de[1]',
 'pcm_fftMag_mfcc_de[2]',
 'pcm_fftMag_mfcc_de[3]',
 'pcm_fftMag_mfcc_de[4]',
 'pcm_fftMag_mfcc_de[5]',
 'pcm_fftMag_mfcc_de[6]',
 'pcm_fftMag_mfcc_de[7]',
 'pcm_fftMag_mfcc_de[8]',
 'pcm_fftMag_mfcc_de[9]',
 'pcm_fftMag_mfcc_de[10]',
 'pcm_fftMag_mfcc_de[11]',
 'pcm_fftMag_mfcc_de[12]',
 'pcm_fftMag_mfcc_de_de[0]',
 'pcm_fftMag_mfcc_de_de[1]',
 'pcm_fftMag_mfcc_de_de[2]',
 'pcm_fftMag_mfcc_de_de[3]',
 'pcm_fftMag_mfcc_de_de[4]',
 'pcm_fftMag_mfcc_de_de[5]',
 'pcm_fftMag_mfcc_de_de[6]',
 'pcm_fftMag_mfcc_de_de[7]',
 'pcm_fftMag_mfcc_de_de[8]',
 'pcm_fftMag_mfcc_de_de[9]',
 'pcm_fftMag_mfcc_de_de[10]',
 'pcm_fftMag_mfcc_de_de[11]',
 'pcm_fftMag_mfcc_de_de[12]\n',
 'chroma[0]',
 'chroma[1]',
 'chroma[2]',
 'chroma[3]',
 'chroma[4]',
 'chroma[5]',
 'chroma[6]',
 'chroma[7]',
 'chroma[8]',
 'chroma[9]',
 'chroma[10]',
 'chroma[11]\n',
 'chroma[0].1',
 'chroma[1].1',
 'chroma[2].1',
 'chroma[3].1',
 'chroma[4].1',
 'chroma[5].1',
 'chroma[6].1',
 'chroma[7].1',
 'chroma[8].1',
 'chroma[9].1',
 'chroma[10].1',
 'chroma[11]\n.1',
 'pcm_LOGenergy\n',
 'voiceProb_sma',
 'F0_sma',
 'pcm_loudness_sma\n',
 'F0final_sma',
 'voicingFinalUnclipped_sma',
 'pcm_loudness_sma\n.1']

In [52]:
sample_df_norm = sample_df[features]
sample_df_norm = (sample_df_norm - sample_df_norm.mean())/(sample_df_norm.max()-sample_df_norm.min())

In [55]:
sample_df_norm['frameTime'] = sample_df['frameTime']

In [56]:
seg_dfs = []
for index in start_segment.index:
    tdf = sample_df_norm[(sample_df_norm.frameTime > int(start_segment[index])) & (sample_df_norm.frameTime < int(end_segment[index])) ]
    seg_dfs.append(tdf)

In [57]:
segment_means = []
for segment in seg_dfs:
    segment_means.append(segment[features].mean())

In [60]:
andy_df.head()

,video,laughter_start,laughter_end,laughter_value,start_segment_s,end_segment_s,start_segment_frame,end_segment_frame
33,AndyWoodhull2,2.0,5.0,1,0.0,2.0,1,60
34,AndyWoodhull2,26.0,27.0,1,5.0,26.0,151,780
35,AndyWoodhull2,31.0,32.0,1,27.0,31.0,811,930
36,AndyWoodhull2,38.0,43.0,1,32.0,38.0,961,1140
37,AndyWoodhull2,49.0,50.0,1,43.0,49.0,1291,1470


In [62]:
segement_values = pd.DataFrame(segment_means,index = andy_df.index)

In [66]:
pd.concat([andy_df,segement_values],axis=1)

,video,laughter_start,laughter_end,laughter_value,start_segment_s,end_segment_s,start_segment_frame,end_segment_frame,pcm_fftMag_mfcc[0],pcm_fftMag_mfcc[1],...,chroma[9].1,chroma[10].1,chroma[11] .1,pcm_LOGenergy,voiceProb_sma,F0_sma,pcm_loudness_sma,F0final_sma,voicingFinalUnclipped_sma,pcm_loudness_sma .1
33,AndyWoodhull2,2.0,5.0,1,0.0,2.0,1,60,-0.061113,-0.083815,...,-0.052289,-0.021384,0.006735,-0.007725,0.087395,0.137038,0.042136,0.069199,0.024203,0.041495
34,AndyWoodhull2,26.0,27.0,1,5.0,26.0,151,780,0.011961,0.057028,...,-0.017121,-0.025124,-0.028390,-0.029596,0.025029,-0.015166,-0.071246,-0.039563,0.002085,-0.071383
35,AndyWoodhull2,31.0,32.0,1,27.0,31.0,811,930,-0.017961,-0.017823,...,0.003479,-0.011363,0.015006,0.028666,0.021452,0.030580,0.066416,0.046529,0.015744,0.065354
36,AndyWoodhull2,38.0,43.0,1,32.0,38.0,961,1140,0.026597,0.027695,...,0.009288,0.012478,0.053472,0.011090,0.117614,0.100898,0.027611,0.027859,0.050224,0.027306
37,AndyWoodhull2,49.0,50.0,1,43.0,49.0,1291,1470,0.002811,-0.005096,...,0.018160,0.026692,0.024414,0.016020,0.069244,0.054964,0.028966,0.003715,0.037430,0.029049
38,AndyWoodhull2,54.0,55.0,1,50.0,54.0,1501,1620,-0.000007,0.048309,...,-0.021687,0.021845,0.055076,0.008677,0.145455,0.145872,0.016009,0.051252,0.050351,0.016409
39,AndyWoodhull2,62.0,64.0,1,55.0,62.0,1651,1860,-0.013366,-0.007803,...,0.009710,0.041498,0.027341,0.011139,0.033344,0.027878,-0.000287,0.016351,0.018786,-0.000398
40,AndyWoodhull2,83.0,84.0,1,64.0,83.0,1921,2490,0.021615,0.070734,...,-0.009250,-0.025658,-0.020368,-0.037810,0.002589,-0.033354,-0.070759,-0.060682,-0.030193,-0.070849
41,AndyWoodhull2,88.0,89.0,1,84.0,88.0,2521,2640,-0.020223,-0.054074,...,-0.032498,-0.020770,-0.005858,0.031964,0.043299,0.051522,0.054609,0.068811,0.033870,0.054342
42,AndyWoodhull2,100.0,102.0,1,89.0,100.0,2671,3000,0.003342,0.081981,...,0.001386,0.031121,0.028407,-0.021059,0.046200,0.040089,-0.045045,-0.014030,0.001086,-0.044808
